<h2>Blum-Goldwasser Probabilistic Encryption</h2>

In [3]:
import numpy 
import math 
import random
import time

In [90]:
#Umbral a partir del cual consideramos que un numero es "grande"
umbral = 2
#Rango de valores a partir del umbral que se pueden generar
rango = 2000

p, q, a, b = generarClavePrivada(umbral, rango)

clavePrivada = [p, q, a, b]
clavePublica = p*q

print("Clave privada --> ", clavePrivada)
print("Clave publica --> ", clavePublica)

msjClaro = leerFichero("msjClaro.txt")
alf = leerFichero("alf.txt")
msjCifrado = encriptar(msjClaro, alf, clavePublica)
print(msjCifrado)

#TODO esta comentado porque he hardcodeado la funcion desencriptar para testear
msjDescrifrado = desencriptar(p, q, a, b, msjCifrado, alf)
print(msjDescrifrado)

Clave privada -->  [911, 1051, -518, 449]
Clave publica -->  957461
[2, 9, 2, 7, 4, 13, 5, 13, 4, 5, 3, 7, 2, 8, 6, 9, 8, 6, 14, 5, 2, 0, 3, 9, 4, 14, 8, 12, 12, 11, 8, 5, 7, 2, 0, 4, 1, 5, 7, 9, 10, 10, 3, 7, 5, 13, 4, 7, 0, 7, 15, 4, 7, 13, 5, 13, 201670]
8 533 25 269
Estoy simulando un sistema MIXTO


In [91]:
def generarClavePrivada(umbral, rango):
    p=0; q=0

    while not cumpleCongruencia(p, 3, 4) or not esPrimo(p):
        p = random.randint(umbral, umbral+rango)

    while not cumpleCongruencia(q, 3, 4)  or p==q or not esPrimo(q): 
        q = random.randint(umbral, umbral+rango)

    a, b = calcularAlgEuclidesExtendido(p, q)
    
    return p, q, a, b

In [84]:
def esPrimo(n):
    for i in range(2,int(n**0.5)+1):
        if n%i==0:
            return False

    return True

In [6]:
def cumpleCongruencia(num, numCongruente, modulo):
    return (num%modulo) == (numCongruente%modulo)

In [7]:
def calcularAlgEuclidesExtendido(p, q):
    restos = [p,q]
    listaA = [1,0]
    listaB = [0,1]
    cocientes = [0,0]

    i=2

    while(restos[i-1] > 1):
        cocientes.append(restos[i-2]//restos[i-1])
        restos.append(restos[i-2]%restos[i-1])
        listaA.append(listaA[i-2] - cocientes[i] * listaA[i-1])
        listaB.append(listaB[i-2] - cocientes[i] * listaB[i-1])
        i += 1

    if restos.pop()==0:
        return listaB.pop(), 0
    
    return listaA.pop(), listaB.pop()


In [8]:
def leerFichero(ruta):
    fichero = open(ruta, "r", encoding="utf8")
    texto = fichero.read()
    fichero.close()
    return texto

In [46]:
def encriptar(msjClaro, alf, n):

    k = math.floor(math.log(n,2))
    #h es la longitud de los bloques en los que se dividira el mensaje binario
    h = math.floor(math.log(k,2))
    x = []
    c = []

    msjBinario = strToBinary(msjClaro, alf)
    
    bloquesMsjBinario = separarBloques(msjBinario, h)
    t = len(bloquesMsjBinario)
    #print("k: ", k, " h: ", h)
    #print(bloquesMsjBinario)

    x.append(inicializarX0(n))

    for i in range(1, t+1):
        xi = pow(x[i-1], 2, n)
        #print("xi: ", xi)
        x.append(xi)

        p = decimalToBinary(x[i], 0)
        p = p[len(p)-h:]
        #print("p: ", p)

        ci = int(bloquesMsjBinario[i-1],2)^int(p,2)
        #print("ci: ", ci)
        c.append(ci)
    
    c.append(pow(x[len(x)-1], 2, n))

    return c

In [64]:
def desencriptar(p, q, a ,b, msjCifrado, alf):
#def desencriptar():
    #OJO el mensaje cifrado se recibe como una lista de numeros
 #   p=499; q=547; a=-57; b=52; t=5; h=18; msjCifrado = [2,0,12,14,4,139680]
    
    #IMPORTANTE, El orden en el que se ejecutan estas lineas esta puesto a posta sino peta, ademas quite t de los parametros que le llegan
    #y lo calculo aqui, porque asi no hace falta que lo devuelva el encriptar
    k = math.floor(math.log(p*q,2))
    h = math.floor(math.log(k,2))

    lastX = msjCifrado.pop()
    msjBinarioBloque = []
    for i in msjCifrado:
        msjBinarioBloque.append(decimalToBinary(i,h))
    t = len(msjBinarioBloque)
    
    d1 = pow((p+1)//4, t+1, p-1)
    d2 = pow((q+1)//4, t+1, q-1)
    
    u = pow(lastX, d1, p)
    v = pow(lastX, d2, q)
    n=p*q

    print(d1,d2,u,v)

    x = [(v*a*p+u*b*q)%(n)]
    m = []

    for i in range(1, t+1):
        xi = pow(x[i-1],2,n)
        #print("xi: ", xi)
        x.append(xi)
            
        p = decimalToBinary(x[i], 0)
        p = p[len(p)-h:]
        #print("p: ", p)
        
        #print("MSJ: ", msjBinarioBloque[i-1])
        m.append(int(p,2)^int(msjBinarioBloque[i-1],2))
        
    #cambiamos a binario la lista de ints que nos devuelve m. importante que al cambiarlo sea de longitud h el binario
    binario=[]
    for i in m:
        binario.append(decimalToBinary(i,h))
    
    #traducimos a string
    return binaryToString(binario, alf)


In [81]:
def binaryToString(binario, alf):
    #nos llega una lista con binarios de long h, se junta todo y se separa en bloques de la longitud que depende del alfabeto
    longBinaria = math.ceil(math.log(len(alf),2)) 
    cadena = ''
    simbolos = list(alf)
    binarioStream = ''
    for i in binario:
        binarioStream = binarioStream + i
    
    binarioBloque = separarBloques(binarioStream, longBinaria)
    #Se traduce cada bloque que representa una posicion y se coge la letra correspondiente
    for i in binarioBloque:
        cadena = cadena + simbolos[int(i,2)]
    return cadena

In [11]:
desencriptar()

TypeError: desencriptar() missing 7 required positional arguments: 'p', 'q', 'a', 'b', 't', 'msjCifrado', and 'alf'

In [19]:
def strToBinary(cadena, alf):
    #numero minimo de digitos binarios para representar todas las letras del alfabeto
    longBinaria = math.ceil(math.log(len(alf),2)) 
    cadenaBinaria = ''

    for caracter in cadena:
        cadenaBinaria += decimalToBinary(alf.find(caracter), longBinaria)

    return cadenaBinaria



In [13]:
def decimalToBinary(caracterNumerico, longBinaria):
    bloqueBinario = bin(caracterNumerico)[2:]

    if(longBinaria==0):
        longBinaria = len(bloqueBinario)

    #Rellena con 0s a la izquierda 
    while(len(bloqueBinario)!=longBinaria):
        bloqueBinario = '0' + bloqueBinario
    
    return bloqueBinario


In [14]:
def inicializarX0(n):
    r = random.randint(1, n-1)
    while not tieneInverso(r, n):
        r = random.randint(1, n-1)
    
    return pow(r, 2, n)

In [15]:
def tieneInverso(numero, modulo):
    
    try:
        pow(numero,-1,modulo)
        return True
    except:
        return False
        
    return False

In [16]:
def separarBloques(texto, longitudBloque):
    temp = list(texto) #Copia
    bloques = []

    while(temp):
        bloque = ''
        for i in range(longitudBloque):
            if(len(temp)>0):
                bloque = temp.pop() + bloque
            else:
                bloque = '0' + bloque
        bloques.insert(0, bloque)

    return bloques